## Election Generator

This notebook generates fake elections that simulate the racial dynamics of an election, at the precinct (or lowest cohesive voting bloc) level.

Given:
1. the number of people in the precinct: $n$
2. the list of candidates in the district: $C = \{c_0, c_1, \dots, c_a\}$, where $a$ is the number of candidates in the race
3. the racial breakdown of the district: $R = \{r_0, r_1, \dots, r_b\}$, where $b$ is the number of races represented. $r_i$ is the fraction of $n$ that race $i$ represents in the district
4. the probabilities that members of certain races vote for certain candidates: $\beta_{i, c_j}$, where $i$ is the race and $c_j$ is the candidate

The election generated has the following information:
1. the number of people that voted for each candidate
2. the racial breakdown of each candidate's voters

In [46]:
from typing import Dict, List, Tuple
import random
from scipy.stats import binom
import numpy as np

The following function interprets $beta_{i, c_j}$ to be the proportion of a racial group that votes for a particular candidate.

In [21]:
def generate_deterministic_election(n: int, candidates: List[str], racial_breakdown: List[float], beta: List[List[float]]) -> Dict[str, Tuple[int, float]]:
    racial_numbers = [round(r_i * n) for r_i in racial_breakdown]
    result = dict()
    for cand_index, candidate in enumerate(candidates):
        racial_result = [r_n * beta[race][cand_index] for race, r_n in enumerate(racial_numbers)]
        num_votes = sum(racial_result)
        result[candidate] = num_votes, racial_result
    return result

In [56]:
generate_deterministic_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]])

[600, 300, 100]


{'a': (360.0, [240.0, 90.0, 30.0]),
 'b': (410.0, [300.0, 60.0, 50.0]),
 'c': (230.0, [60.0, 150.0, 20.0])}

The following function interprets $beta_{i, c_j}$ to be the probability that a single member of a racial group turns out to vote for a particular candidate.

It has an extra paramter, `whole_ouput` that rounds the output to the nearest whole number if it is set to `True`.

In [59]:
def generate_simple_random_election(n: int, candidates: List[str], racial_breakdown: List[float], beta: List[List[float]], whole_output=False) -> Dict[str, Tuple[int, float]]:
    racial_numbers = [round(r_i * n) for r_i in racial_breakdown]
    result = dict()
    for cand_index, candidate in enumerate(candidates):
        racial_result = [binom.rvs(n=r_n, p=beta[race][cand_index], size=100).mean() for race, r_n in enumerate(racial_numbers)]
        num_votes = sum(racial_result)
        if whole_output:
            num_votes = round(num_votes)
            racial_result = np.round(racial_result).tolist()
        result[candidate] = num_votes, racial_result
    return result

In [60]:
generate_simple_random_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]])

{'a': (358.8, [239.38, 89.75, 29.67]),
 'b': (410.09, [300.56, 59.46, 50.07]),
 'c': (231.53999999999996, [61.01, 150.89, 19.64])}

In [61]:
generate_simple_random_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]], whole_output=True)

{'a': (361.0, [241.0, 90.0, 30.0]),
 'b': (410.0, [299.0, 60.0, 51.0]),
 'c': (230.0, [60.0, 150.0, 20.0])}

In [70]:
def generate_random_election(n: int, candidates: List[str], racial_breakdown: List[float], beta: List[List[float]], whole_output=False) -> Dict[str, Tuple[int, float]]:
    racial_numbers = [round(r_i * n) for r_i in racial_breakdown]
    num_races = len(racial_breakdown)
    
    result = {'a': (0, np.zeros(num_races)), 'b': (0, np.zeros(num_races)), 'c': (0, np.zeros(num_races))}
    
    for cand_index, candidate in enumerate(candidates):
        for race, r_n in enumerate(racial_numbers):
            for voter in range(r_n):
                vote = np.random.choice(candidates, beta[race][cand_index])
                prev_total, prev_breakdown = result[vote]
                prev_breakdown[race] += 1
                result[vote] = prev_total + 1, prev_breakdown
    return result

In [71]:
generate_random_election(1000, ['a', 'b', 'c'], [0.6, 0.3, 0.1], [[0.4, 0.5, 0.1], [0.3, 0.2, 0.5], [0.3, 0.5, 0.2]])

TypeError: 'float' object cannot be interpreted as an integer